In [1]:
using LinearAlgebra
using BlackBoxOptim
using HDF5

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [23]:
function infidelity_norm(ρ, σ)
    (1 - real(tr(ρ * σ)) - 2*sqrt(abs(det(ρ))*abs(det(σ))))^2
end

infidelity_norm (generic function with 1 method)

In [24]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [4]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

initial_params .+= 1;

In [5]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [6]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γ = "0.25133"
γᶠ = parse(ComplexF64, γ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γ);

In [30]:
# Setup the BlackBoxOptim optimizer
res = bboptimize(objective, 
SearchRange = (-26.0, 26.0), 
NumDimensions = length(initial_params), 
Method = :generating_set_search, #:adaptive_de_rand_1_bin_radiuslimited
MaxSteps = 1000, #500
autodiff = :forward,
TraceMode = :verbose)

Starting optimization with optimizer GeneratingSetSearcher(BlackBoxOptim.ConstantDirectionGen)
0.00 secs, 1 evals, 0 steps, fitness=141157.203517281
2.67 secs, 6 evals, 1 steps, fitness=141144.910147793
5.34 secs, 11 evals, 2 steps, fitness=141083.309281730
5.86 secs, 12 evals, 3 steps, fitness=140935.716267173
9.59 secs, 19 evals, 4 steps, fitness=119302.176429609
10.13 secs, 20 evals, 5 steps, fitness=109652.089189551
10.66 secs, 21 evals, 6 steps, fitness=101321.363347559
11.20 secs, 22 evals, 7 steps, fitness=80811.483905164
12.26 secs, 24 evals, 8 steps, fitness=71210.470135777
15.47 secs, 30 evals, 9 steps, fitness=62807.248746135
17.64 secs, 34 evals, 10 steps, fitness=58921.791240763
19.79 secs, 38 evals, 11 steps, fitness=58900.979407178
21.38 secs, 41 evals, 12 steps, fitness=58779.008821211
22.46 secs, 43 evals, 13 steps, fitness=57115.320398867
23.52 secs, 45 evals, 14 steps, fitness=55821.364940526
25.67 secs, 49 evals, 15 steps, fitness=55737.762137650
27.80 secs, 53 eval

BlackBoxOptim.OptimizationResults("generating_set_search", "Max number of steps (1000) reached", 1001, 1.722265862329058e9, 15110.01800107956, ParamsDictChain[ParamsDictChain[Dict{Symbol, Any}(:RngSeed => 813613, :NumDimensions => 21, :autodiff => :forward, :SearchRange => (-26.0, 26.0), :TraceMode => :verbose, :Method => :generating_set_search, :MaxSteps => 1000),Dict{Symbol, Any}()],Dict{Symbol, Any}(:CallbackInterval => -1.0, :TargetFitness => nothing, :TraceMode => :compact, :FitnessScheme => ScalarFitnessScheme{true}(), :MinDeltaFitnessTolerance => 1.0e-50, :NumDimensions => :NotSpecified, :FitnessTolerance => 1.0e-8, :TraceInterval => 0.5, :MaxStepsWithoutProgress => 10000, :MaxSteps => 10000…)], 28076, ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64, Vector{Float64}}(90.87531331085894, [26.0, 11.464154827425741, 19.946041225272538, 5.459604548094401, 2.3220744127285506, -3.745711828892727, 4.211463137022633, -2.217075136250667, -2.4624703792879146, -4.707

In [11]:
# Setup the BlackBoxOptim optimizer
res = bboptimize(objective, 
SearchRange = (-26.0, 26.0), 
NumDimensions = length(initial_params), 
Method = :generating_set_search, #:adaptive_de_rand_1_bin_radiuslimited
MaxSteps = 10000, #500
autodiff = :forward,
TraceMode = :verbose)

Starting optimization with optimizer GeneratingSetSearcher(BlackBoxOptim.ConstantDirectionGen)
0.00 secs, 1 evals, 0 steps, fitness=1818676.779871122
0.74 secs, 3 evals, 2 steps, fitness=1321624.041686805
1.48 secs, 5 evals, 4 steps, fitness=595621.516005996
2.58 secs, 8 evals, 5 steps, fitness=595615.294887825
4.06 secs, 12 evals, 6 steps, fitness=82351.284378540
5.17 secs, 15 evals, 7 steps, fitness=82336.128032750
6.66 secs, 19 evals, 8 steps, fitness=69978.698537913
7.41 secs, 21 evals, 9 steps, fitness=69978.638680929
10.04 secs, 28 evals, 10 steps, fitness=58963.714107365
10.78 secs, 30 evals, 11 steps, fitness=55864.953415265
11.91 secs, 33 evals, 12 steps, fitness=55861.096327457
13.02 secs, 36 evals, 13 steps, fitness=48816.383550837
14.87 secs, 41 evals, 14 steps, fitness=48793.665026615
15.99 secs, 44 evals, 15 steps, fitness=48674.389890149
18.20 secs, 50 evals, 16 steps, fitness=48627.911031809
19.31 secs, 53 evals, 17 steps, fitness=48595.557002289
21.17 secs, 58 evals, 1

Excessive output truncated after 524291 bytes.

BlackBoxOptim.OptimizationResults("generating_set_search", "Max number of steps (10000) reached", 10001, 1.722002786192642e9, 110866.98214006424, ParamsDictChain[ParamsDictChain[Dict{Symbol, Any}(:RngSeed => 242141, :NumDimensions => 21, :SearchRange => (-25.0, 25.0), :TraceMode => :verbose, :Method => :generating_set_search, :MaxSteps => 10000),Dict{Symbol, Any}()],Dict{Symbol, Any}(:CallbackInterval => -1.0, :TargetFitness => nothing, :TraceMode => :compact, :FitnessScheme => ScalarFitnessScheme{true}(), :MinDeltaFitnessTolerance => 1.0e-50, :NumDimensions => :NotSpecified, :FitnessTolerance => 1.0e-8, :TraceInterval => 0.5, :MaxStepsWithoutProgress => 10000, :MaxSteps => 10000…)], 274825, ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64, Vector{Float64}}(1.0935615592015619e-8, [25.0, -0.0035773901809861286, 0.0011745659829962563, -0.19601275300507837, 0.030844494829988633, 0.005465813348799031, -0.14884171896739318, 0.35221065693579945, -0.009021205585849401, 

In [26]:
# Extract optimized parameters
optimized_params = best_candidate(res)

# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

Optimized H: ComplexF64[12.5 + 0.0im 9.366156770919137 - 4.576884251735663im; 9.366156770919137 + 4.576884251735663im -12.5 + 0.0im]
Optimized C: ComplexF64[82.97236772298362 + 0.0im -3.0622302643517267 + 86.43462870880762im -12.762917362427919 + 57.21963267448209im; -3.0622302643517267 - 86.43462870880762im 91.81286094954037 + 0.0im 61.23686661387857 + 11.424986942725281im; -12.762917362427919 - 57.21963267448209im 61.23686661387857 - 11.424986942725281im 42.27315233964868 + 0.0im]


In [27]:
optimized_H

2×2 Matrix{ComplexF64}:
    12.5+0.0im      9.36616-4.57688im
 9.36616+4.57688im    -12.5+0.0im

In [28]:
optimized_H

2×2 Matrix{ComplexF64}:
    12.5+0.0im      9.36616-4.57688im
 9.36616+4.57688im    -12.5+0.0im

In [29]:
optimized_C

3×3 Matrix{ComplexF64}:
  82.9724+0.0im      -3.06223+86.4346im  -12.7629+57.2196im
 -3.06223-86.4346im   91.8129+0.0im       61.2369+11.425im
 -12.7629-57.2196im   61.2369-11.425im    42.2732+0.0im

In [9]:
optimized_C

3×3 Matrix{ComplexF64}:
  2454.08+0.0im       38.6602+2449.37im  -76.4486-15.0218im
  38.6602-2449.37im   2446.43+0.0im      -19.7415+78.9891im
 -76.4486+15.0218im  -19.7415-78.9891im   27.3426+0.0im

In [ ]:
search_range = (-25.0, 25.0)

# List of methods to compare
methods = [
    #:adaptive_de_rand_1_bin_radiuslimited,
    #:adaptive_de_rand_1_bin,
    #:de_rand_1_bin_radiuslimited,
    #:de_rand_1_bin,
    #:de_best_1_bin,
    #:de_rand_2_bin,
    #:de_best_2_bin,
    #:pbasis_de_rand_2_bin_radiuslimited,
    #:pbasis_de_rand_2_bin,
    #:adaptive_de_rand_2_bin_radiuslimited,
    #:adaptive_de_rand_2_bin,
    #:sep_de_rand_1_bin,
    #:sep_de_rand_2_bin,
    #:sep_de_best_1_bin,
    #:sep_de_best_2_bin,
    #:multi_strategy_adaptive_de,
    #:multi_de_rand_1_bin,
    #:multi_de_best_1_bin,
    #:multi_de_rand_2_bin,
    #:multi_de_best_2_bin,
    #:best_1_bin_radiuslimited,
    #:rand_1_bin_radiuslimited,
    #:rand_2_bin_radiuslimited,
    #:best_2_bin_radiuslimited,
    #:best_1_bin,
    #:rand_1_bin,
    #:rand_2_bin,
    #:best_2_bin,
    #:self_adaptive_de_rand_1_bin,
    #:self_adaptive_de_rand_2_bin,
    #:self_adaptive_de_best_1_bin,
    #:self_adaptive_de_best_2_bin,
    #:gradient_de_rand_1_bin,
    #:gradient_de_rand_2_bin,
    #:gradient_de_best_1_bin,
    #:gradient_de_best_2_bin
]

In [ ]:
res_compare = compare_optimizers(objective; SearchRange = (-26, 26),
 NumDimensions =length(initial_params),
 #methods=methods,
 max_steps=10000,
 autodiff = :forward
 #MaxTime = 25.0
 );

In [ ]:
# Extract optimized parameters
optimized_params = best_candidate(res_compare)

# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

In [ ]:
result_bbo = bboptimize(polyGEXY; SearchRange = (-26.0, 26.0), NumDimensions = 21,
    MaxTime = 20, 
    Method = :adaptive_de_rand_1_bin_radiuslimited,
    InitialPopulation = [initial_guess])

optimal_value_bbo = best_candidate(result_bbo)
minimum_function_value_bbo = best_fitness(result_bbo)

In [15]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

get_lindblad_operators (generic function with 1 method)

In [16]:
Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
Cˢⁱᵈ = convert.(ComplexF64,optimized_C)

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

1-element Vector{Any}:
 ComplexF64[0.011981351862763726 + 0.0im -0.35021940212826963 - 0.3116345455691105im; -0.04265282394735939 - 0.048760448432132425im -0.011981351862763726 + 0.0im]

In [17]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [18]:
using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


In [20]:
γᵢ= γ

FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9767457828782189
Median fidelity for 0.25133: 0.9894426574763489
